# Azure AI Agents met ondersteuning voor Model Context Protocol (MCP)

Deze notebook laat zien hoe je Azure AI Agents kunt gebruiken met Model Context Protocol (MCP)-tools om een intelligent agent te creëren die externe MCP-servers kan benutten voor verbeterde mogelijkheden.


## Vereiste NuGet-pakketten installeren

Eerst moeten we het Azure AI Agents Persistent-pakket installeren, dat de kernfunctionaliteit biedt voor het werken met Azure AI Agents.


## Voordelen van Keyless Authenticatie

Dit notebook demonstreert **keyless authenticatie**, wat verschillende voordelen biedt:
- ✅ **Geen API-sleutels om te beheren** - Maakt gebruik van Azure identity-based authenticatie
- ✅ **Verbeterde beveiliging** - Geen geheimen opgeslagen in code of configuratie
- ✅ **Automatische credentialrotatie** - Azure beheert de levenscyclus van credentials
- ✅ **Rolgebaseerde toegang** - Maakt gebruik van Azure RBAC voor gedetailleerde machtigingen

De `DefaultAzureCredential` zal automatisch de beste beschikbare bron voor credentials gebruiken:
1. Managed Identity (bij uitvoering in Azure)
2. Azure CLI-credentials (tijdens ontwikkeling)
3. Visual Studio-credentials
4. Omgevingsvariabelen (indien geconfigureerd)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Installeer het Azure Identity-pakket voor authenticatie met Azure-services met behulp van DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Vereiste namespaces importeren

Importeer de benodigde namespaces voor Azure AI Agents en Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent Client configureren (Keyless Authentication)

Stel de configuratievariabelen in en maak de PersistentAgentsClient aan met **keyless authentication**:
- **projectEndpoint**: Het Azure AI Foundry projectendpoint
- **modelDeploymentName**: De naam van het geïmplementeerde AI-model (GPT-4.1 nano)
- **mcpServerUrl**: De URL van de MCP-server (Microsoft Learn API)
- **mcpServerLabel**: Een label om de MCP-server te identificeren
- **DefaultAzureCredential**: Maakt gebruik van managed identity, Azure CLI of andere bronnen voor authenticatie (geen API-sleutels vereist)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Maak MCP Tool Definitie

Maak een MCP-tooldefinitie die verbinding maakt met de Microsoft Learn MCP-server. Hiermee kan de agent toegang krijgen tot Microsoft Learn-inhoud en documentatie.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Een AI-agent maken

Maak een permanente AI-agent met het opgegeven model en MCP-tools. De agent is geconfigureerd met:
- Het GPT-4.1 nano-model
- Instructies om MCP-tools te gebruiken voor ondersteuning
- Toegang tot de Microsoft Learn MCP-server


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Maak een gespreksthread en stuur een bericht

Maak een gespreksthread aan en stuur een gebruikersbericht waarin je vraagt naar het verschil tussen Azure OpenAI en OpenAI. Dit zal de vaardigheid van de agent testen om de MCP-tools te gebruiken om nauwkeurige informatie te geven.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Configureer MCP Tool Resources (Keyless)

Stel de MCP-toolresources in. Voor een echt keyless aanpak kun je aangepaste headers verwijderen als de MCP-server Azure-identiteitsgebaseerde authenticatie ondersteunt. Het onderstaande voorbeeld laat zien hoe je headers kunt toevoegen indien nodig, maar voor keyless scenario's zijn deze mogelijk niet vereist.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Start Agent Run

Maak en start een run om het bericht van de gebruiker te verwerken. De agent zal de geconfigureerde MCP-tools en -bronnen gebruiken om een reactie te genereren.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Bewaak Run en Behandel Toolgoedkeuringen (Keyless)

Houd de status van de agent-run in de gaten en handel eventuele vereiste toolgoedkeuringen af. Deze loop:
1. Wacht tot de run voltooid is of actie vereist
2. Keurt MCP-tooloproepen automatisch goed wanneer nodig
3. Voor keyless authenticatie zijn headers mogelijk niet nodig als de MCP-server Azure-identiteit ondersteunt


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Resultaten van het Gesprek Weergeven

Haal alle berichten in de thread op en toon ze, inclusief zowel de vraag van de gebruiker als het antwoord van de agent. De berichten worden weergegeven in chronologische volgorde met tijdstempels en rolindicatoren.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
